# Figure 23 Data Generation

This notebook processes raw two-point correlation experimental data to generate processed data for Figure 1.

## Process:
1. Load raw experimental data from UUIDs (high and low field)
2. Process data (extract chi1 and chi12 correlation parameters)
3. Save processed data for plotting


In [23]:
# Import common utilities
import sys
from pathlib import Path
import importlib

# Add common directory to path (works in Jupyter notebooks)
common_path = Path().resolve().parent / 'common_scripts'
sys.path.insert(0, str(common_path))
import correlation_fun
from raw_data_loader import load_raw_data_by_uuid
from data_saver import save_figure_data
import data_processor as processor
import numpy as np

# Reload modules to ensure latest changes
importlib.reload(processor)
importlib.reload(correlation_fun)


data_path = Path().resolve().parent / 'data'
# Data path

figure_path = Path().resolve().parent / 'figures'


# Alias for convenience
load_by_uuid = load_raw_data_by_uuid


## Load Raw Data

Load raw experimental data for two-point correlation measurements (high and low field).


In [3]:
# Low field data UUIDs
# S1: Reference position
# S2: Single-dot measurements at different positions
# S12: Two-point correlation measurements
low_field_uuids = {
    "S1": "1730672289460108893",  # Reference position
    "S2": "1730750147026108893",  # Single-dot measurements
    "S12": "1730827675550108893"  # Two-point correlations (CORRECTED)
}

# High field data UUIDs (75mV)
high_field_uuids = {
    "S1": "1727225883960108893",  # Reference position
    "S2": "1727226751312108893",  # Single-dot measurements
    "S12": "1727224984279108893"  # Two-point correlations
}

# Load low field data
print("Loading low field data...")
data_low = {}
for key, uuid in low_field_uuids.items():
    dataset = load_by_uuid(uuid, data_path)
    if dataset is None:
        print(f"Warning: Failed to load {key} with UUID {uuid}")
    else:
        data_low[key] = dataset
        print(f"✓ Loaded {key}")

# Load high field data
print("\nLoading high field data...")
data_high = {}
for key, uuid in high_field_uuids.items():
    dataset = load_by_uuid(uuid, data_path)
    if dataset is None:
        print(f"Warning: Failed to load {key} with UUID {uuid}")
    else:
        data_high[key] = dataset
        print(f"✓ Loaded {key}")


Loading low field data...
✓ Loaded S1
✓ Loaded S2
✓ Loaded S12

Loading high field data...
✓ Loaded S1
✓ Loaded S2
✓ Loaded S12


In [4]:
# Extract low field data
# Low field uses m1_3() measurement channel (based on original notebook)
S1_low = data_low["S1"].m1_3()    # Reference position measurements
S2_low = data_low["S2"].m1_3()    # Shuttled position measurements
S12_low = data_low["S12"].m1_3()  # Cross-correlation measurements

# Extract time arrays
wait_times12 = data_low["S12"].m1_3.y()      # Wait times for S12 and S2
wait_times1 = data_low["S2"].m1_3.y()
wait_times_ref_low = data_low["S1"].m1_3.x()   # Wait times for reference S1

# Calculate shuttling distances in nm
# Formula: (20 - position) * 0.06 * 180
shuttling_distances_low = (20 - data_low["S12"].m1_3.x()) * 0.06 * 180

# Use wait_times12 for S12 and S2, wait_times_ref_low for S1
wait_times_low = wait_times1

print(f"Low field data shapes:")
print(f"  S1 (reference): {S1_low.shape}")
print(f"  S2 (single-dot): {S2_low.shape}")
print(f"  S12 (two-point): {S12_low.shape}")
print(f"  Wait times (S12/S2): {wait_times_low.shape}")
print(f"  Wait times (S1 ref): {wait_times_ref_low.shape}")
print(f"  Positions: {len(shuttling_distances_low)}")
print(f"  Shuttling positions: {data_low['S12'].m1_3.x()}")


Low field data shapes:
  S1 (reference): (200,)
  S2 (single-dot): (18, 180)
  S12 (two-point): (18, 180)
  Wait times (S12/S2): (180,)
  Wait times (S1 ref): (200,)
  Positions: 18
  Shuttling positions: [ 0.          1.05882353  2.11764706  3.17647059  4.23529412  5.29411765
  6.35294118  7.41176471  8.47058824  9.52941176 10.58823529 11.64705882
 12.70588235 13.76470588 14.82352941 15.88235294 16.94117647 18.        ]


In [5]:
importlib.reload(processor)

# Process low field: chi1 (single-dot) parameters
params_chi1_low = []
guess = {
    "T2": 5000,
    "n": 1.5,
    "freq": 1e-2
}

data1_ref = S1_low.copy()
if len(data1_ref.shape) > 1:
    data1_ref = data1_ref[0]
data1_ref = data1_ref - np.mean(data1_ref)

A, A_err, T2, T2_err, n, n_err, t_data, envelope, y_detrended, phi, f, f_err, B = \
    processor.get_fit_hilbert_drift(data1_ref, wait_times_ref_low, guess, use_hilbert=False)
sig = 1 / np.sqrt(2) / np.pi / T2
params_chi1_low.append({
    'A': A, 'T2': T2, 'n': n, 'phi': phi, 'freq': f, 'B': B,
    'sig': sig, 'dsig': T2_err / np.sqrt(2) / np.pi / T2**2,
    'dT': T2_err, 'dn': n_err, 'dfreq': f_err, 'x': 0,
    'envelope': envelope, 't_data': t_data
})

for i in range(S2_low.shape[0]):
    if len(S2_low.shape) == 2:
        data2 = S2_low[i, :].copy()
    else:
        data2 = S2_low[i].copy()
    data2 = data2 - np.mean(data2)
    
    A, A_err, T2, T2_err, n, n_err, t_data, envelope, y_detrended, phi, f, f_err, B = \
        processor.get_fit_hilbert_drift(data2, wait_times_low, guess, use_hilbert=False)
    sig = 1 / np.sqrt(2) / np.pi / T2
    params_chi1_low.append({
        'A': A, 'T2': T2, 'n': n, 'phi': phi, 'freq': f, 'B': B,
        'sig': sig, 'dsig': T2_err / np.sqrt(2) / np.pi / T2**2,
        'dT': T2_err, 'dn': n_err, 'dfreq': f_err, 'x': shuttling_distances_low[i],
        'envelope': envelope, 't_data': t_data
    })

print(f"Processed {len(params_chi1_low)} chi1 (single-dot) positions")




Processed 19 chi1 (single-dot) positions


In [6]:
# Process low field: chi12 (two-point) parameters
params_chi12_low = []

for i in range(S12_low.shape[0]):
    if len(S12_low.shape) == 2:
        data12 = S12_low[i, :].copy()
    else:
        data12 = S12_low[i].copy()
    data12 = data12 - np.mean(data12)
    
    T2_xn = params_chi1_low[0]["T2"]
    alpha_xn = params_chi1_low[0]["n"]
    T2_xm = params_chi1_low[i+1]["T2"]
    alpha_xm = params_chi1_low[i+1]["n"]
  
    A, A_err, T2, T2_err, n, n_err, t_data, envelope, y_detrended, phi, f, f_err, B = \
        processor.get_fit_hilbert_drift(data12, wait_times12, guess, use_hilbert=False)
    
    T12eff, alpha12eff = correlation_fun.calculate_chi_with_background_corrected(
        [T2_xn, T2_xm], [alpha_xn, alpha_xm], 2, np.eye(2), 1e99, 2
    )
    
    sig = 1 / np.sqrt(2) / np.pi / T2
    rnm, rnm_err, chi_dc_fit = processor.fit_rnm(wait_times12, 2*envelope/A, T2_xn, T2_xm, alpha_xn, alpha_xm, 0, [-2,2])
    
    params_chi12_low.append({
        'A': A, 'T2': T2, 'n': n, 'phi': phi, 'freq': f, 'B': B,
        'sig': sig, 'dsig': T2_err / np.sqrt(2) / np.pi / T2**2,
        'dT': T2_err, 'dn': n_err, 'dfreq': f_err, 'x': shuttling_distances_low[i],
        'rnm': rnm, 'rnm_err': rnm_err, 'T12eff': T12eff, 'alpha12eff': alpha12eff,
        'envelope': envelope, 't_data': t_data
    })

print(f"Processed {len(params_chi12_low)} chi12 (two-point) positions")

params_chi12_low = sorted(params_chi12_low, key=lambda p: p['x'])
params_chi1_low = sorted(params_chi1_low, key=lambda d: d['x'])
params_low = {"chi1": params_chi1_low, "chi12": params_chi12_low}




Processed 18 chi12 (two-point) positions


In [7]:
# Extract high field data
m1_S1_high = data_high["S1"].m1_5()
m1_S2_high = data_high["S2"].m1_5()
m1_S12_high = data_high["S12"].m1_5()

S1_high = m1_S1_high[0, 0]
S2_high = m1_S2_high[0]
S12_high = m1_S12_high[0]

wait_times_high = data_high["S12"].m1_5.k()
wait_times_ref_high = data_high["S1"].m1_5.k()

CV_times = np.array([1., 6.28571429, 11.57142857, 16.85714286, 22.14285714, 27.42857143, 32.71428571, 38.])
shuttling_distances_high = 216 - CV_times * 0.03 * 180

print(f"High field data shapes:")
print(f"  S1 (reference): {S1_high.shape}")
print(f"  S2 (single-dot): {S2_high.shape}")
print(f"  S12 (two-point): {S12_high.shape}")
print(f"  Wait times (S12/S2): {wait_times_high.shape}")
print(f"  Wait times (S1 ref): {wait_times_ref_high.shape}")
print(f"  Positions: {len(shuttling_distances_high)}")


High field data shapes:
  S1 (reference): (70,)
  S2 (single-dot): (8, 70)
  S12 (two-point): (8, 70)
  Wait times (S12/S2): (70,)
  Wait times (S1 ref): (70,)
  Positions: 8


In [8]:
# Process high field: chi1 (single-dot) parameters
params_chi1_high = []

data1_ref = S1_high.copy()
data1_ref -= np.mean(data1_ref)
guess_ref = {"T2": 3000, "n": 1.5, "freq": 0.01}
A, A_err, T2, T2_err, n, n_err, t_data, y_detrended, envelope, phi, f, f_err, B = \
    processor.get_fit_hilbert_drift(data1_ref, wait_times_ref_high, guess_ref, use_hilbert=True, window_size=10)
sig = 1 / np.sqrt(2) / np.pi / T2
params_chi1_high.append({
    'A': A, 'T2': T2, 'n': n, 'phi': phi, 'freq': f, 'B': B,
    'sig': sig, 'dsig': T2_err / np.sqrt(2) / np.pi / T2**2,
    'dT': T2_err, 'dn': n_err, 'dfreq': f_err, 'x': 0,
    'envelope': envelope, 't_data': t_data
})

for i in range(S2_high.shape[0]):
    data2 = S2_high[i, :].copy()
    data2 -= np.mean(data2)
    guess = {"T2": 4000, "n": 2, "freq": 0.01}
    A, A_err, T2, T2_err, n, n_err, t_data, y_detrended, envelope, phi, f, f_err, B = \
        processor.get_fit_hilbert_drift(data2, wait_times_high, guess, use_hilbert=True, window_size=20)
    sig = 1 / np.sqrt(2) / np.pi / T2
    params_chi1_high.append({
        'A': A, 'T2': T2, 'n': n, 'phi': phi, 'freq': f, 'B': B,
        'sig': sig, 'dsig': T2_err / np.sqrt(2) / np.pi / T2**2,
        'dT': T2_err, 'dn': n_err, 'dfreq': f_err, 'x': shuttling_distances_high[i],
        'envelope': envelope, 't_data': t_data
    })




In [9]:
# Process high field: chi12 (two-point) parameters
params_chi12_high = []

for i in range(S12_high.shape[0]):
    data12 = S12_high[i, :].copy()
    data12 -= np.mean(data12)
    
    T2_xn = params_chi1_high[0]["T2"]
    alpha_xn = params_chi1_high[0]["n"]
    T2_xm = params_chi1_high[i+1]["T2"]
    alpha_xm = params_chi1_high[i+1]["n"]
    freq = params_chi1_high[0]["freq"]
    
    guess = {"T2": 4000, "n": 2, "freq": 0.01}
    A, A_err, T2, T2_err, n, n_err, t_data, envelope, y_detrended, phi, f, f_err, B = \
        processor.get_fit_hilbert_drift(data12, wait_times_high, guess, use_hilbert=True, window_size=20)
    
    T12eff, alpha12eff = correlation_fun.calculate_chi_with_background_corrected(
        [T2_xn, T2_xm], [alpha_xn, alpha_xm], 2, np.eye(2), 1e99, 2
    )
    
    sig = 1 / np.sqrt(2) / np.pi / T2
    rnm, rnm_err, chi_dc_fit = processor.fit_rnm(wait_times_high, 2*envelope/A, T2_xn, T2_xm, alpha_xn, alpha_xm, 0, [-2,2])

    params_chi12_high.append({
        'A': A, 'T2': T2, 'n': n, 'phi': phi, 'freq': f, 'B': B,
        'sig': sig, 'dsig': T2_err / np.sqrt(2) / np.pi / T2**2,
        'dT': T2_err, 'dn': n_err, 'dfreq': f_err, 'x': shuttling_distances_high[i],
        'rnm': rnm, 'rnm_err': rnm_err, 'T12eff': T12eff, 'alpha12eff': alpha12eff,
        'envelope': envelope, 't_data': t_data
    })

print(f"Processed {len(params_chi12_high)} chi12 (two-point) positions")

params_chi1_high = sorted(params_chi1_high, key=lambda d: d['x'])
params_chi12_high = sorted(params_chi12_high, key=lambda d: d['x'])
params_high = {"chi1": params_chi1_high, "chi12": params_chi12_high}



Processed 8 chi12 (two-point) positions


## Saving processed data

In [ ]:
# Save low field data
save_figure_data(
    params_low,
    figure_number="fig2_3",
    filename="fig23_low.pkl",
    metadata={
        "field": "low",
        "description": "Two-point correlation parameters for low field",
        "shuttling_distances": shuttling_distances_low.tolist()
    }
)

# Save high field data (75mV)
save_figure_data(
    params_high,
    figure_number="fig2_3",
    filename="fig23_high_75mV.pkl",
    metadata={
        "field": "high",
        "voltage": "75mV",
        "description": "Two-point correlation parameters for high field (75mV)",
        "shuttling_distances": shuttling_distances_high.tolist()
    }
)

print("✓ Data saved successfully!")


✓ Saved fig23 data to /Users/krzywdaja/Documents/spatial-correlations-conveyor/data_analysis/protection_code_repo/processed_data/fig23/fig23_low.pkl
  Metadata saved to /Users/krzywdaja/Documents/spatial-correlations-conveyor/data_analysis/protection_code_repo/processed_data/fig23/fig23_low.json
✓ Saved fig23 data to /Users/krzywdaja/Documents/spatial-correlations-conveyor/data_analysis/protection_code_repo/processed_data/fig23/fig23_high_75mV.pkl
  Metadata saved to /Users/krzywdaja/Documents/spatial-correlations-conveyor/data_analysis/protection_code_repo/processed_data/fig23/fig23_high_75mV.json
✓ Data saved successfully!


In [ ]:
import json
import os

# Gather all relevant data for high field comprehensive plot
data_to_save = {
    "params_high": params_high,
    "shuttling_distances_high": shuttling_distances_high,
    "S1_high": S1_high.tolist() if hasattr(S1_high, "tolist") else S1_high,
    "S2_high": S2_high.tolist() if hasattr(S2_high, "tolist") else S2_high,
    "S12_high": S12_high.tolist() if hasattr(S12_high, "tolist") else S12_high,
    "wait_times_high": wait_times_high.tolist() if hasattr(wait_times_high, "tolist") else wait_times_high,
    "wait_times_ref_high": wait_times_ref_high.tolist() if hasattr(wait_times_ref_high, "tolist") else wait_times_ref_high,
}

save_dir = os.path.join("..", "processed_data", "fig19_20")
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "data_high.json")

# Convert any numpy types to native python types so JSON serializes cleanly
def make_json_serializable(d):
    import numpy as np
    if isinstance(d, dict):
        return {k: make_json_serializable(v) for k, v in d.items()}
    elif isinstance(d, np.ndarray):
        return d.tolist()
    elif isinstance(d, (np.integer, np.floating)):
        return d.item()
    elif isinstance(d, list):
        return [make_json_serializable(x) for x in d]
    else:
        return d

with open(save_path, "w") as f:
    json.dump(make_json_serializable(data_to_save), f, indent=2)

print(f"Saved high-field comprehensive plot data to {save_path}")


In [ ]:
import json
import os

# Gather all relevant data for low field comprehensive plot
data_to_save = {
    "params_low": params_low,
    "shuttling_distances_low": shuttling_distances_low,
    "S1_low": S1_low.tolist() if hasattr(S1_low, "tolist") else S1_low,
    "S2_low": S2_low.tolist() if hasattr(S2_low, "tolist") else S2_low,
    "S12_low": S12_low.tolist() if hasattr(S12_low, "tolist") else S12_low,
    "wait_times_low": wait_times_low.tolist() if hasattr(wait_times_low, "tolist") else wait_times_low,
    "wait_times_ref_low": wait_times_ref_low.tolist() if hasattr(wait_times_ref_low, "tolist") else wait_times_ref_low,
    "wait_times12": wait_times12.tolist() if hasattr(wait_times12, "tolist") else wait_times12,
}

save_dir = os.path.join("..", "processed_data", "fig19_20")
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "data_low.json")

# Convert any numpy types to native python types so JSON serializes cleanly
def make_json_serializable(d):
    import numpy as np
    if isinstance(d, dict):
        return {k: make_json_serializable(v) for k, v in d.items()}
    elif isinstance(d, np.ndarray):
        return d.tolist()
    elif isinstance(d, (np.integer, np.floating)):
        return d.item()
    elif isinstance(d, list):
        return [make_json_serializable(x) for x in d]
    else:
        return d

with open(save_path, "w") as f:
    json.dump(make_json_serializable(data_to_save), f, indent=2)

print(f"Saved low-field comprehensive plot data to {save_path}")


## Variability study



In [24]:
# Import plotting utilities
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


In [17]:
# Load data for 65mV, 75mV, 85mV
data_65mv = {"S12": {"id": 1727222345743108893}, 
             "S1": {"id": 1727223245897108893},
             "S2": {"id": 1727224112791108893}}

data_75mv = {"S12": {"id": 1727224984279108893},
             "S1": {"id": 1727225883960108893}, 
             "S2": {"id": 1727226751312108893}}

data_85mv = {"S12": {"id": 1727227620625108893},
             "S1": {"id": 1727228520034108893}, 
             "S2": {"id": 1727229387657108893}}

voltage_data_configs = [data_65mv, data_75mv, data_85mv]
voltage_labels = ['65mV', '75mV', '85mV']

# Load all voltage data
print("Loading data for 65mV, 75mV, 85mV...")
all_voltage_data = []
for m, config in enumerate(voltage_data_configs):
    print(f"\nLoading {voltage_labels[m]} data...")
    voltage_data = {}
    for key, uuid_dict in config.items():
        uuid = uuid_dict["id"]
        dataset = load_by_uuid(uuid, data_path)
        if dataset is None:
            print(f"Warning: Failed to load {key} with UUID {uuid}")
        else:
            voltage_data[key] = dataset
            print(f"✓ Loaded {key}")
    all_voltage_data.append(voltage_data)


Loading data for 65mV, 75mV, 85mV...

Loading 65mV data...
✓ Loaded S12
✓ Loaded S1
✓ Loaded S2

Loading 75mV data...
✓ Loaded S12
✓ Loaded S1
✓ Loaded S2

Loading 85mV data...
✓ Loaded S12
✓ Loaded S1
✓ Loaded S2


In [18]:
# Process data for each voltage using high-field processing approach
params_all_voltages = []

for m, voltage_data in enumerate(all_voltage_data):
    print(f"\nProcessing {voltage_labels[m]} data...")
    
    # Extract high field data (same approach as cell 9)
    m1_S1 = voltage_data["S1"].m1_5()
    m1_S2 = voltage_data["S2"].m1_5()
    m1_S12 = voltage_data["S12"].m1_5()

    S1 = m1_S1[0, 0]
    S2 = m1_S2[0]
    S12 = m1_S12[0]

    wait_times = voltage_data["S12"].m1_5.k()
    wait_times_ref = voltage_data["S1"].m1_5.k()

    CV_times = np.array([1., 6.28571429, 11.57142857, 16.85714286, 22.14285714, 27.42857143, 32.71428571, 38.])
    shuttling_distances = 216 - CV_times * 0.03 * 180

    print(f"  Data shapes: S1={S1.shape}, S2={S2.shape}, S12={S12.shape}")
    print(f"  Positions: {len(shuttling_distances)}")
    
    # Process chi1 (single-dot) parameters (same approach as cell 11)
    params_chi1 = []

    data1_ref = S1.copy()
    data1_ref -= np.mean(data1_ref)
    guess_ref = {"T2": 3000, "n": 2, "freq": 0.01}
    A, A_err, T2, T2_err, n, n_err, t_data, y_detrended, envelope, phi, f, f_err, B = \
        processor.get_fit_hilbert_drift(data1_ref, wait_times_ref, guess_ref, use_hilbert=True, window_size=10)
    sig = 1 / np.sqrt(2) / np.pi / T2
    params_chi1.append({
        'A': A, 'T2': T2, 'n': n, 'phi': phi, 'freq': f, 'B': B,
        'sig': sig, 'dsig': T2_err / np.sqrt(2) / np.pi / T2**2,
        'dT': T2_err, 'dn': n_err, 'dfreq': f_err, 'x': 0,
        'envelope': envelope, 't_data': t_data
    })

    for i in range(S2.shape[0]):
        data2 = S2[i, :].copy()
        data2 -= np.mean(data2)
        guess = {"T2": 4000, "n": 2, "freq": 0.01}
        A, A_err, T2, T2_err, n, n_err, t_data, y_detrended, envelope, phi, f, f_err, B = \
            processor.get_fit_hilbert_drift(data2, wait_times, guess, use_hilbert=True, window_size=20)
        sig = 1 / np.sqrt(2) / np.pi / T2
        params_chi1.append({
            'A': A, 'T2': T2, 'n': n, 'phi': phi, 'freq': f, 'B': B,
            'sig': sig, 'dsig': T2_err / np.sqrt(2) / np.pi / T2**2,
            'dT': T2_err, 'dn': n_err, 'dfreq': f_err, 'x': shuttling_distances[i],
            'envelope': envelope, 't_data': t_data
        })
 
    print(f"  Processed {len(params_chi1)} chi1 positions")
    
    # Process chi12 (two-point) parameters (same approach as cell 14)
    params_chi12 = []

    for i in range(S12.shape[0]):
        data12 = S12[i, :].copy()
        data12 -= np.mean(data12)
        
        T2_xn = params_chi1[0]["T2"]
        alpha_xn = params_chi1[0]["n"]
        T2_xm = params_chi1[i+1]["T2"]
        alpha_xm = params_chi1[i+1]["n"]
        freq = params_chi1[0]["freq"]
        
        guess = {"T2": 4000, "n": 2, "freq": 0.01}
        A, A_err, T2, T2_err, n, n_err, t_data, envelope, y_detrended, phi, f, f_err, B = \
            processor.get_fit_hilbert_drift(data12, wait_times, guess, use_hilbert=True, window_size=20)
        
        T12eff, alpha12eff = correlation_fun.calculate_chi_with_background_corrected(
            [T2_xn, T2_xm], [alpha_xn, alpha_xm], 2, np.eye(2), 1e99, 2
        )
        
        sig = 1 / np.sqrt(2) / np.pi / T2
        rnm, rnm_err, chi_dc_fit = processor.fit_rnm(wait_times, 2*envelope/A, T2_xn, T2_xm, alpha_xn, alpha_xm, 0, [-2,2])

        params_chi12.append({
            'A': A, 'T2': T2, 'n': n, 'phi': phi, 'freq': f, 'B': B,
            'sig': sig, 'dsig': T2_err / np.sqrt(2) / np.pi / T2**2,
            'dT': T2_err, 'dn': n_err, 'dfreq': f_err, 'x': shuttling_distances[i],
            'rnm': rnm, 'rnm_err': rnm_err, 'T12eff': T12eff, 'alpha12eff': alpha12eff,
            'envelope': envelope, 't_data': t_data
        })


    print(f"  Processed {len(params_chi12)} chi12 positions")

    params_chi1 = sorted(params_chi1, key=lambda d: d['x'])
    params_chi12 = sorted(params_chi12, key=lambda d: d['x'])
    params_voltage = {"chi1": params_chi1, "chi12": params_chi12}
    params_all_voltages.append(params_voltage)

print(f"\n✓ Processed all {len(params_all_voltages)} voltages")



Processing 65mV data...
  Data shapes: S1=(70,), S2=(8, 70), S12=(8, 70)
  Positions: 8
  Processed 9 chi1 positions
  Processed 8 chi12 positions

Processing 75mV data...
  Data shapes: S1=(70,), S2=(8, 70), S12=(8, 70)
  Positions: 8
  Processed 9 chi1 positions
  Processed 8 chi12 positions

Processing 85mV data...
  Data shapes: S1=(70,), S2=(8, 70), S12=(8, 70)
  Positions: 8
  Processed 9 chi1 positions
  Processed 8 chi12 positions

✓ Processed all 3 voltages


In [22]:
# Save processed data for Figure 18 (voltage comparison plot)
import json
from pathlib import Path

# Create output directory
processed_data_path = Path().resolve().parent / 'processed_data' / 'fig18'
processed_data_path.mkdir(parents=True, exist_ok=True)

# Convert any numpy types to native python types so JSON serializes cleanly
def make_json_serializable(d):
    import numpy as np
    if isinstance(d, dict):
        return {k: make_json_serializable(v) for k, v in d.items()}
    elif isinstance(d, np.ndarray):
        return d.tolist()
    elif isinstance(d, (np.integer, np.floating)):
        return d.item()
    elif isinstance(d, list):
        return [make_json_serializable(x) for x in d]
    else:
        return d

# Prepare data to save
data_to_save = {
    "params_all_voltages": params_all_voltages,
    "voltage_labels": voltage_labels,  # ['65mV', '75mV', '85mV']
    "description": "Processed two-point correlation parameters for three voltages (65mV, 75mV, 85mV) for Figure 18",
    "note": "Each voltage has chi1 (9 positions) and chi12 (8 positions) parameters"
}

# Save to JSON
save_path = processed_data_path / 'data.json'
with open(save_path, "w") as f:
    json.dump(make_json_serializable(data_to_save), f, indent=2)

print(f"✓ Saved Figure 18 data to {save_path}")
print(f"  Number of voltages: {len(params_all_voltages)}")
print(f"  Voltage labels: {voltage_labels}")


✓ Saved Figure 18 data to /Users/krzywdaja/Documents/spatial-correlations-conveyor/data_analysis/protection_code_repo/processed_data/fig18/data.json
  Number of voltages: 3
  Voltage labels: ['65mV', '75mV', '85mV']
